#Imports

In [1]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.linear_model import LinearRegression

#Daten einlesen

In [2]:
raw_data = pd.read_csv('12411-0005.csv')
raw_data

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf6 in position 31: invalid start byte

In [3]:
raw_data = pd.read_csv('12411-0005.csv', skiprows=6,  encoding='ISO-8859-1', delimiter=';')
raw_data

,Unnamed: 0,31.12.2017,31.12.2018,31.12.2019,31.12.2020,31.12.2021
0,unter 1 Jahr,785074.0,783978.0,774870.0,769380.0,791254.0
1,1-Jährige,796149.0,796374.0,794132.0,783593.0,780795.0
2,2-Jährige,770628.0,802651.0,802415.0,798366.0,789145.0
3,3-Jährige,760868.0,776763.0,807816.0,806198.0,803334.0
4,4-Jährige,734059.0,766631.0,782143.0,811601.0,810805.0
...,...,...,...,...,...,...
86,Insgesamt,82792351.0,83019213.0,83166711.0,83155031.0,83237124.0
87,__________,NaN,NaN,NaN,NaN,NaN
88,Bis 1989: Früheres Bundesgebiet\n\nAb 2011: Er...,NaN,NaN,NaN,NaN,NaN
89,"© Statistisches Bundesamt (Destatis), 2022",NaN,NaN,NaN,NaN,NaN


In [6]:
raw_data = pd.read_csv('12411-0005.csv', skiprows=6,  nrows=87, encoding='ISO-8859-1', sep=';')
raw_data

,Unnamed: 0,31.12.2017,31.12.2018,31.12.2019,31.12.2020,31.12.2021
0,unter 1 Jahr,785074,783978,774870,769380,791254
1,1-Jährige,796149,796374,794132,783593,780795
2,2-Jährige,770628,802651,802415,798366,789145
3,3-Jährige,760868,776763,807816,806198,803334
4,4-Jährige,734059,766631,782143,811601,810805
...,...,...,...,...,...,...
82,82-Jährige,601037,622557,643708,687458,732760
83,83-Jährige,521955,563565,585337,604464,644569
84,84-Jährige,389704,484835,524879,544669,561973
85,85 Jahre und mehr,2265473,2277509,2386854,2505932,2625444


#Daten kennenlernen und bereinigen

In [13]:
# eine Splate umbennen 
raw_data1 = raw_data.rename(columns= {'Unnamed: 0' : 'Altersjahre'})
raw_data1

,Altersjahre,31.12.2017,31.12.2018,31.12.2019,31.12.2020,31.12.2021
0,unter 1 Jahr,785074,783978,774870,769380,791254
1,1-Jährige,796149,796374,794132,783593,780795
2,2-Jährige,770628,802651,802415,798366,789145
3,3-Jährige,760868,776763,807816,806198,803334
4,4-Jährige,734059,766631,782143,811601,810805
...,...,...,...,...,...,...
82,82-Jährige,601037,622557,643708,687458,732760
83,83-Jährige,521955,563565,585337,604464,644569
84,84-Jährige,389704,484835,524879,544669,561973
85,85 Jahre und mehr,2265473,2277509,2386854,2505932,2625444


In [14]:
raw_data1.shape

(87, 6)

In [15]:
raw_data1.index

RangeIndex(start=0, stop=87, step=1)

In [16]:
raw_data1.dtypes

Altersjahre    object
31.12.2017      int64
31.12.2018      int64
31.12.2019      int64
31.12.2020      int64
31.12.2021      int64
dtype: object

In [17]:
raw_data1.columns

Index(['Altersjahre', '31.12.2017', '31.12.2018', '31.12.2019', '31.12.2020',
       '31.12.2021'],
      dtype='object')

In [18]:
raw_data1.describe()

,31.12.2017,31.12.2018,31.12.2019,31.12.2020,31.12.2021
count,8.700000e+01,8.700000e+01,8.700000e+01,8.700000e+01,8.700000e+01
mean,1.903272e+06,1.908488e+06,1.911878e+06,1.911610e+06,1.913497e+06
std,8.776871e+06,8.800792e+06,8.816537e+06,8.815488e+06,8.824384e+06
min,3.897040e+05,4.848350e+05,5.248790e+05,5.446690e+05,5.300530e+05
25%,7.748180e+05,7.665740e+05,7.717085e+05,7.750195e+05,7.768890e+05
50%,9.522320e+05,9.533140e+05,9.476460e+05,9.547930e+05,9.542340e+05
75%,1.084220e+06,1.091325e+06,1.088328e+06,1.083692e+06,1.083802e+06
max,8.279235e+07,8.301921e+07,8.316671e+07,8.315503e+07,8.323712e+07


In [19]:
raw_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Altersjahre  87 non-null     object
 1   31.12.2017   87 non-null     int64 
 2   31.12.2018   87 non-null     int64 
 3   31.12.2019   87 non-null     int64 
 4   31.12.2020   87 non-null     int64 
 5   31.12.2021   87 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 4.2+ KB
